In [2]:
from dashboard.logic.io import GSHEETS_URL, read_gsheet, comment_button
import pandas as pd
import numpy as np

df = read_gsheet(
    GSHEETS_URL, 
    header=None
)


In [26]:
def findRowColRegex(df: pd.DataFrame, pat: str, case: bool=True, regex:bool=True):
    
    # make sure only one such pat exists in the df
    df_mask = df.apply(lambda x: x.str.contains(pat, case=case, regex=regex) if x.dtype == 'object' else None)
    
    count = df_mask.sum().sum()
    
    if count < 1: 
        raise ValueError(f'Given {pat} did not give any results.')
    if count > 1:
        raise ValueError(f'Given {pat} gave more than 1 results.')
     
    # col and row values where pat
    row = df_mask.any(axis='columns').argmax()
    col = df_mask.any(axis='index').argmax()
    
    return row, col

def getDataFrames(df: pd.DataFrame):

    df = df.reset_index(drop=True)
    dataframes_dict = {}
    
    # MAIN DF
    main_df_idx2 = (df == 'Monthly Income').any(axis='columns').argmax()
    dataframes_dict['main'] = df.iloc[:main_df_idx2+1,:3].dropna(how='all', axis='rows')
    
    # ADS DF
    r1, c1 = findRowColRegex(df, "My Finance Course")
    
    
    ads_df = df.iloc[r1:,c1:].reset_index(drop=True)
    ads_df = (ads_df
        .loc[:(ads_df[c1].isna()).argmax()-1, :]
        .dropna(how='all', axis='columns')
    )
    dataframes_dict['ads'] = ads_df
    
    # OTHER DFS
    df_names = [
    'announcements',
    'advice',
    'error_warning',
    'risk',
    'skip',
    'skip',
    'skip',
    'historical',
    'cash_pos',
    'general_notes',
    'success',
    ]
    
    # Find rows after main df where all values are NaN-s.
    nan_rows = df.iloc[main_df_idx2+1:,].isna().all(axis='columns')
    
    # Indices where all NaN-s in the row
    dfs_idxs = []
    
    for i,bool in nan_rows.items():
        
        if i == nan_rows.index[-1]:
            dfs_idxs.append(None)
            break
        if bool:
            dfs_idxs.append(i)
    
    # raw dataframes
    dfs = []        
    for i,idx in enumerate(dfs_idxs):
        if i == len(dfs_idxs) - 1:
            break
        dfs.append(df.iloc[idx:dfs_idxs[i+1]])
    
    # strip dataframes from NaN-s and add to dictionary
    for df_, name in zip(dfs, df_names):
        
        if name == 'skip': continue
        
        dataframes_dict[name] = (df_
            .dropna(how='all', axis='columns')
            .dropna(how='all', axis='rows')
        )
    
    return dataframes_dict

df1 = df.copy()

#findRowColRegex(df1, 'My Finance Course')

df_dict = getDataFrames(df1)
df_dict.keys()

dict_keys(['main', 'ads', 'announcements', 'advice', 'error_warning', 'risk', 'historical', 'cash_pos', 'general_notes', 'success'])

In [53]:
df_dict['error_warning'].values[]

array([['NOTE: Occasionally the external websites where this data is pulled from will be down, meaning the allocation % below will not be accurate']],
      dtype=object)

In [64]:
(df_dict['main'] == '#ERROR!').sum().sum()

2

In [1]:
from dashboard.data.investments import df_table

df1 = df_table.copy()
(df1 == '#ERROR!').sum().sum()

1

In [8]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook
from bokeh.palettes import Category10
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.models import ColumnDataSource, HoverTool, Legend, LabelSet, LegendItem
from bokeh.resources import CDN
from bokeh.embed import components

In [5]:
output_notebook()

Loading BokehJS ...

In [11]:
df1 = df.copy()
df1 = df1.drop("Comments", axis='columns').query("`Asset Class` != 'Monthly Income' & `Total Value` != '#ERROR!'")
df1['Total Value'] = df1['Total Value'].replace(r"[\$,]", "", regex=True).astype(float)
df1['angle'] = df1["Total Value"] / df1["Total Value"].sum() * 2*np.pi
df1 = df1.rename(columns={"Total Value": "total_value"})

df1

,Asset Class,total_value,angle
1,Gold/Silver (physical),236815.54,0.413628
2,Gold/Silver (allocated),1216701.98,2.125125
3,Business Equity,432375.75,0.755199
4,Real Estate Equity (IOM/UK),1179431.85,2.060028
5,Real Estate Equity (US),482000.00,0.841874
6,Stocks,0.00,0.000000
7,Cash,50000.00,0.087331


In [12]:
df2 = df1.copy()
df2['cumsum_start'] = df2['angle'].cumsum(axis='rows').shift(1).fillna(0)
df2['cumsum_end'] = df2['angle'].cumsum(axis='rows')
df2 = df2.rename(columns={"Asset Class": 'asset_class'})
df2 = df2.query("asset_class != 'Total (USD)'")
df2

,asset_class,total_value,angle,cumsum_start,cumsum_end
1,Gold/Silver (physical),236815.54,0.413628,0.000000,0.413628
2,Gold/Silver (allocated),1216701.98,2.125125,0.413628,2.538753
3,Business Equity,432375.75,0.755199,2.538753,3.293952
4,Real Estate Equity (IOM/UK),1179431.85,2.060028,3.293952,5.353980
5,Real Estate Equity (US),482000.00,0.841874,5.353980,6.195854
6,Stocks,0.00,0.000000,6.195854,6.195854
7,Cash,50000.00,0.087331,6.195854,6.283185


In [13]:
def pie_chart(
    df: pd.DataFrame, 
    x: str, 
    y: str, 
    radius: float=0.8,
    x_range: tuple[float, float]=(-1, 1.0),
    percentage_decimal: int=1,
    fig_height: int=350,
    background_color: str='#212529',
    pallette: dict=Category10,
    hover_tooltip: str='default',
    legend_place: str='right',
    fig_kwargs: dict={},
    wedge_kwargs: dict=dict(line_width=1.5, alpha=0.7),
    legend_kwargs: dict=dict(location='center', click_policy="hide",
                             label_text_color='white', border_line_width=0),
    label_kwargs: dict=dict(text_font_size='10pt', text_align='center')
    ):
    
    
    # calculate sector start and end angles
    df['angle'] = df[y] / df[y].sum() * 2 * np.pi
    df['cumsum_start'] = df['angle'].cumsum(axis='rows').shift(1).fillna(0)
    df['cumsum_end'] = df['angle'].cumsum(axis='rows')
    
    # calculate y percentages for hover & labels
    df['percentage_number'] = (df[y] / df[y].sum() * 100).round(percentage_decimal)
    df['percentage_hover'] = df['percentage_number'].astype(str)
    df['percentage_label'] = df['percentage_number'].apply(lambda x: "" if x < 5 else f"{x}%")
    
    # project label text coordinates to polar coordinates
    df['label_x_pos'] = np.cos(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    df['label_y_pos'] = np.sin(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    
    # remove assets that are 0
    df = df[df[y] > 0]
    
    # reset dataframe index to start with 0
    df = df.reset_index(drop=True)
    
    # init the figure/canvas for the plot
    p = figure(height=fig_height, toolbar_location=None, x_range=x_range, **fig_kwargs)
    
    legend_items = []
    for idx, color in enumerate(pallette[df.shape[0]]):
        
        source = ColumnDataSource(df.iloc[idx,:].to_frame().T)
        
        # create the glyphs renderers
        wedge = p.wedge(x=0, y=0, radius=radius, start_angle="cumsum_start", 
                        end_angle="cumsum_end", source=source, **wedge_kwargs,
                        fill_color=color, hover_fill_color=color,
                        line_color=background_color, hover_line_color=background_color,
                        line_alpha=1, hover_alpha=1, hover_line_alpha=1)
        
        label = LabelSet(x='label_x_pos', y='label_y_pos', text='percentage_label',
                         source=source, level='glyph', text_color=background_color, **label_kwargs)
        
        hover_tooltip = hover_tooltip if hover_tooltip != 'default' else \
            f"""
                <div>
                    <p style="margin:0;font-weight:bold;color:grey;">@{x}</p>
                    <p style="padding:0;margin:0;font-weight:bold;">@{y}{{$0,0.00}} (@percentage_hover%)</p>
                </div>
            """
        
        p.add_layout(label)
        p.add_tools(HoverTool(renderers=[wedge],
                              tooltips=hover_tooltip))

        legend_items.append(LegendItem(label=df[x][idx], renderers=[wedge]))
    
    # legend
    legend = Legend(items=legend_items, **legend_kwargs,
                    background_fill_color=background_color) 
    p.add_layout(legend, place=legend_place)
    
    # figure attributes
    p.toolbar.active_drag = None
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    
    p.min_border=0
    p.outline_line_alpha=0
    p.outline_line_width=0
    p.outline_line_color = p.background_fill_color = p.border_fill_color = background_color

    return p

show(pie_chart(df2, 'asset_class', 'total_value'))

In [3]:
from dashboard.data.investments import df_plot


df_plot

# show(pie_chart(df_plot, 'Asset_Class', 'Total_Value'))

,Asset_Class,Total_Value
1,Gold/Silver (physical),236815.54
2,Gold/Silver (allocated),1216701.98
3,Business Equity,432375.75
4,Real Estate Equity (IOM/UK),1179431.85
5,Real Estate Equity (US),482000.00
6,Stocks,0.00
7,Cash,50000.00


In [88]:



def pie_chart(
    df: pd.DataFrame, 
    x: str, 
    y: str, 
    radius: float=0.8,
    x_range: tuple[float, float]=(-1, 1.0),
    percentage_decimal: int=1,
    fig_height: int=350,
    background_color: str='#212529',
    pallette: dict=Category10,
    hover_tooltip: str='default',
    legend_place: str='right',
    fig_kwargs: dict={},
    wedge_kwargs: dict=dict(line_width=1.5, alpha=0.7),
    legend_kwargs: dict=dict(location='center', click_policy="hide",
                             label_text_color='white', border_line_width=0),
    label_kwargs: dict=dict(text_font_size='10pt', text_align='center')
    ):
    
    # calculate sector start and end angles
    df['angle'] = df[y] / df[y].sum() * 2 * np.pi
    df['cumsum_start'] = df['angle'].cumsum(axis='rows').shift(1).fillna(0)
    df['cumsum_end'] = df['angle'].cumsum(axis='rows')
    
    # calculate y percentages for hover & labels
    df['percentage_number'] = (df[y] / df[y].sum() * 100).round(percentage_decimal)
    df['percentage_hover'] = df['percentage_number'].astype(str)
    df['percentage_label'] = df['percentage_number'].apply(lambda x: "" if x < 5 else f"{x}%")
    
    # project label text coordinates to polar coordinates
    df['label_x_pos'] = np.cos(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    df['label_y_pos'] = np.sin(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    
    # remove assets that are 0
    df = df[df[y] > 0]
    
    # reset dataframe index to start with 0
    df = df.reset_index(drop=True)
    
    # init the figure/canvas for the plot
    p = figure(height=fig_height, toolbar_location=None, x_range=x_range, **fig_kwargs)
    
    legend_items = []
    for idx, color in enumerate(pallette[df.shape[0]]):
        
        source = ColumnDataSource(df.iloc[idx,:].to_frame().T)
        
        # create the glyphs renderers
        wedge = p.wedge(x=0, y=0, radius=radius, start_angle="cumsum_start", 
                        end_angle="cumsum_end", source=source, **wedge_kwargs,
                        fill_color=color, hover_fill_color=color,
                        line_color=background_color, hover_line_color=background_color,
                        line_alpha=1, hover_alpha=1, hover_line_alpha=1)
        
        label = LabelSet(x='label_x_pos', y='label_y_pos', text='percentage_label',
                         source=source, level='glyph', text_color=background_color, **label_kwargs)
        
        hover_tooltip = hover_tooltip if hover_tooltip != 'default' else \
            f"""
                <div>
                    <p style="margin:0;font-weight:bold;color:grey;">@{x}</p>
                    <p style="padding:0;margin:0;font-weight:bold;">@{y}{{$0,0.00}} (@percentage_hover%)</p>
                </div>
            """
        
        p.add_layout(label)
        p.add_tools(HoverTool(renderers=[wedge],
                              tooltips=hover_tooltip))

        legend_items.append(LegendItem(label=df[x][idx], renderers=[wedge]))
    
    # legend
    legend = Legend(items=legend_items, **legend_kwargs,
                    background_fill_color=background_color) 
    p.add_layout(legend, place=legend_place)
    
    # figure attributes
    p.toolbar.active_drag = None
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    
    p.min_border=0
    p.outline_line_alpha=0
    p.outline_line_width=0
    p.outline_line_color = p.background_fill_color = p.border_fill_color = background_color

    return p
    
a = pie_chart(df2, x='asset_class', y='total_value', radius=0.8)

In [ ]:
def bokeh_pie_chart(df, x, y, radius, 
    fig_height=350, 
    pallette=Category10,
    pie_line_color='white',
    legend_place='right',
    **fig_kwargs,
    **wedge_kwargs,
    **legend_kwargs,):
    
    # calculate sector start and end angles
    df['angle'] = df[y] / df[y].sum() * 2 * np.pi
    df['cumsum_start'] = df['angle'].cumsum(axis='rows').shift(1).fillna(0)
    df['cumsum_end'] = df['angle'].cumsum(axis='rows')
    
    # calculate y percentages for hover & labels
    df['percentage_number'] = (df[y] / df[y].sum() * 100).round(1)
    df['percentage_hover'] = df['percentage_number'].map('{:,.1f}%'.format)
    df['percentage_label'] = df['percentage_number'].apply(lambda x: "" if x < 5 else f"{x}%")
    
    # project label text coordinates to polar coordinates
    df['label_x_pos'] = np.cos(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    df['label_y_pos'] = np.sin(df['angle'].cumsum() - df['angle'].div(2)) * 3 * radius/4
    
    # remove assets that are 0
    df = df[df[y] > 0]
    
    # reset dataframe index to start with 0
    df = df.reset_index(drop=True)
    
    main_source = ColumnDataSource(df)
    
    # init the figure/canvas for the plot
    p = figure(height=fig_height, toolbar_location=None, x_range=(-1, 1.0), **fig_kwargs)
    
    legend_items = []
    
    for idx, color in enumerate(pallette[df.shape[0]]):
        
        source = ColumnDataSource(df.iloc[idx,:].to_frame().T)
        
        # create the glyphs on canvas
        wedge = p.wedge(x=0, y=0, radius=radius, start_angle="cumsum_start", 
                        end_angle="cumsum_end", source=source, line_color=pie_line_color, line_width=1.5,
                        fill_color=color, hover_fill_color=color, hover_line_color='#212529',
                        alpha=0.7, hover_alpha=1, line_alpha=1, hover_line_alpha=1, **wedge_kwargs)
        
        label = LabelSet(x='label_x_pos', y='label_y_pos', text='percentage_label',
                         text_font_size='10pt', text_color="black", source=source,
                         text_align='center', text_alpha=0.65, level='glyph')
        
        p.add_layout(label)
        p.add_tools(HoverTool(renderers=[wedge],
                              tooltips=f"""
                              <div>
                                  <p style="margin:0;font-weight:bold;color:grey;">@{x}</p>
                                  <p style="padding:0;margin:0;font-weight:bold;">@{y}{{$0,0.00}} (@percentage_hover)</p>
                              </div>
                              """,
                              mode='mouse'
            )
        )

        legend_items.append(LegendItem(label=df[x][idx], renderers=[wedge]))
    
    # legend
    legend = Legend(items=legend_items, location='center', **legend_kwargs) 
    p.add_layout(legend, place=legend_place)
    p.legend.click_policy="hide"
    p.legend.background_fill_color='#212529'
    p.legend.label_text_color='white'
    p.legend.border_line_width=0
    
    p.toolbar.active_drag = None
    p.axis.axis_label = None
    p.axis.visible = False
    p.grid.grid_line_color = None
    p.outline_line_alpha=0
    p.outline_line_width=0
    p.outline_line_color='#212529'
    p.background_fill_color='#212529'
    p.border_fill_color='#212529'
    p.min_border=0
    
    
    show(p)